# 📚 מדריך אינטראקטיבי: טיפול בערכים חסרים ב-Polars

<div dir="rtl">

## 🎯 מטרת המחברת

במחברת זו תלמדו:
- ✅ איך לזהות ערכים חסרים (null ו-NaN)
- ✅ איך למחוק שורות ועמודות עם ערכים חסרים
- ✅ איך למלא ערכים חסרים בשיטות שונות
- ✅ מתי להשתמש בכל שיטה

**הוראות שימוש:**
1. הריצו כל תא קוד לפי הסדר
2. קראו את ההסברים בתאי Markdown
3. נסו לשנות את הקוד והריצו מחדש
4. השלימו את התרגילים בסוף

בהצלחה! 🚀

</div>

---
## 📦 ייבוא ספריות

ראשית, נייבא את הספריות הנדרשות.

In [ ]:
# ייבוא ספריות
import polars as pl
from datetime import date
import numpy as np

print("✅ הספריות נטענו בהצלחה!")
print(f"גרסת Polars: {pl.__version__}")

---
## 🔍 חלק 1: זיהוי ערכים חסרים

### יצירת נתונים לדוגמה

ניצור DataFrame עם טמפרטורות יומיות שבהן יש ערכים חסרים מסוגים שונים.

In [ ]:
# יצירת טווח תאריכים
date_col = pl.date_range(
    date(2023, 1, 1), 
    date(2023, 1, 15), 
    '1d', 
    eager=True
)

# רשימת טמפרטורות עם ערכים חסרים (None ו-NaN)
avg_temp_c_list = [
    -3, None, 6, -1, np.nan, 
    6, 4, None, 1, 2, 
    np.nan, 7, 9, -2, None
]

# יצירת DataFrame
df = pl.DataFrame({
    'date': date_col,
    'avg_temp_celsius': avg_temp_c_list
}, strict=False)

# הצגת הנתונים
print("הנתונים שנוצרו:")
df

### 📝 תרגיל 1.1: ספירת ערכים חסרים

השלימו את הקוד כדי לספור כמה ערכים חסרים יש בכל עמודה.

In [ ]:
# השלימו את הקוד:
# df.______()

# התשובה שלכם:

<details>
<summary>💡 לחצו כאן לפתרון</summary>

```python
df.null_count()
```
</details>

### 🔍 הבדל בין null ל-NaN

בואו נראה את ההבדל בין שני סוגי הערכים החסרים:

In [ ]:
# ספירת null
null_count = df.select(
    pl.col('avg_temp_celsius').is_null().sum()
).item()

# ספירת NaN
nan_count = df.select(
    pl.col('avg_temp_celsius').is_nan().sum()
).item()

print(f"ערכי null: {null_count}")
print(f"ערכי NaN: {nan_count}")
print(f"סה\"כ ערכים חסרים: {null_count + nan_count}")

### 📝 תרגיל 1.2: סינון שורות

סננו את כל השורות שבהן **יש** ערך חסר (null) בעמודת הטמפרטורה.

In [ ]:
# השלימו את הקוד:
# df.filter(pl.col('avg_temp_celsius').______())

# התשובה שלכם:

---
## 🗑️ חלק 2: מחיקת ערכים חסרים

### שימוש ב-drop_nulls()

In [ ]:
# הצגת גודל לפני מחיקה
print(f"שורות לפני מחיקה: {len(df)}")

# מחיקת שורות עם null
df_no_nulls = df.drop_nulls()

print(f"שורות אחרי מחיקת null: {len(df_no_nulls)}")
print(f"נמחקו: {len(df) - len(df_no_nulls)} שורות")

# האם עדיין יש NaN?
remaining_nan = df_no_nulls.select(
    pl.col('avg_temp_celsius').is_nan().sum()
).item()
print(f"\n⚠️ NaN שנשארו: {remaining_nan}")

### 📝 תרגיל 2.1: מחיקה מלאה

מחקו גם את ערכי ה-NaN (רמז: תחילה המירו NaN ל-null)

In [ ]:
# השלימו:
# df_fully_clean = (
#     df
#     .with_columns(pl.col('avg_temp_celsius').______(None))
#     ._______()
# )

# התשובה שלכם:

---
## 📝 חלק 3: מילוי ערכים חסרים

### הכנה: המרת NaN ל-null

In [ ]:
# המרת NaN ל-null כדי שכל הפונקציות יעבדו
df_clean = df.with_columns(
    pl.col('avg_temp_celsius').fill_nan(None)
)

print("✅ כל הערכים החסרים הם עכשיו null")

### 3.1 מילוי בערך קבוע

In [ ]:
# מילוי באפס
df_clean.with_columns(
    pl.col('avg_temp_celsius').fill_null(0).alias('filled_with_zero')
)

### 3.2 אסטרטגיות סטטיסטיות

בואו נשווה בין אסטרטגיות מילוי שונות:

In [ ]:
df_strategies = df_clean.select(
    'avg_temp_celsius',
    mean_filled=pl.col('avg_temp_celsius').fill_null(strategy='mean'),
    min_filled=pl.col('avg_temp_celsius').fill_null(strategy='min'),
    max_filled=pl.col('avg_temp_celsius').fill_null(strategy='max'),
)

df_strategies

### 📝 תרגיל 3.1: מילוי בחציון

מלאו את הערכים החסרים בחציון (median) של העמודה.

In [ ]:
# רמז: השתמשו ב-fill_null() עם pl.col('...').median()

# התשובה שלכם:

### 3.3 אינטרפולציה

In [ ]:
df_interpolated = df_clean.select(
    'date',
    'avg_temp_celsius',
    linear=pl.col('avg_temp_celsius').interpolate(),
    nearest=pl.col('avg_temp_celsius').interpolate(method='nearest')
)

df_interpolated

---
## 🔄 חלק 4: Forward Fill ו-Backward Fill

### יצירת דוגמה פשוטה

In [ ]:
df_fills = pl.DataFrame({
    'values': [1, 2, None, None, None, 3, 4, None, 5]
})

print("נתונים לדוגמה:")
df_fills

### השוואת שיטות

In [ ]:
df_fills_result = df_fills.select(
    'values',
    forward_fill=pl.col('values').forward_fill(),
    forward_1=pl.col('values').forward_fill(limit=1),
    backward_fill=pl.col('values').backward_fill(),
    backward_2=pl.col('values').backward_fill(limit=2),
)

df_fills_result

### 📝 תרגיל 4.1: זיהוי דפוסים

השלימו את הטבלה:

| שורה | ערך מקורי | forward_fill | backward_fill |
|------|-----------|--------------|---------------|
| 3    | None      | ?            | ?             |
| 4    | None      | ?            | ?             |
| 5    | None      | ?            | ?             |

---
## 🏋️ תרגילים מתקדמים

### תרגיל 5: ניתוח מקיף

In [ ]:
# צרו DataFrame חדש עם נתוני מכירות
sales_data = pl.DataFrame({
    'month': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun'],
    'sales': [1000, None, 1200, None, None, 1500],
    'expenses': [800, 850, None, 900, None, 950]
})

print("נתוני מכירות:")
sales_data

**משימות:**

1. מצאו כמה ערכים חסרים יש בכל עמודה
2. מלאו את עמודת המכירות באינטרפולציה
3. מלאו את עמודת ההוצאות בממוצע
4. חשבו את הרווח (sales - expenses) לכל חודש

In [ ]:
# הפתרון שלכם כאן:

# 1. ספירת nulls


# 2. אינטרפולציה למכירות


# 3. ממוצע להוצאות


# 4. חישוב רווח


---
## 🎓 סיכום

במחברת זו למדתם:

✅ **זיהוי ערכים חסרים:**
- `null_count()` - ספירה מהירה
- `is_null()` - זיהוי שורות ספציפיות
- `is_nan()` - הבחנה בין null ל-NaN

✅ **מחיקת ערכים חסרים:**
- `drop_nulls()` - הסרת שורות
- `fill_nan(None)` + `drop_nulls()` - הסרה מלאה

✅ **מילוי ערכים חסרים:**
- `fill_null()` - ערכים קבועים ואסטרטגיות
- `interpolate()` - אינטרפולציה
- `forward_fill()` / `backward_fill()` - מילוי מכיוון

### 📚 מה הלאה?

- התנסו עם הנתונים שלכם
- קראו את המדריך המקיף
- תרגלו עם datasets אמיתיים
- בדקו את התיעוד הרשמי של Polars

**בהצלחה! 🐻‍❄️**